# Fase 3: Función predict_demand(product_id, date) y preparación para despliegue (30 min)

**Objetivo:**  
- Implementar una función `predict_demand(product_id: str, date: str) -> float` que cargue el modelo entrenado (`rnn_demand_model.h5`), prepare la secuencia de 14 lags para ese producto justo antes de la fecha dada, y devuelva la predicción de ventas.  
- Probar la función con ejemplos reales (al menos 3 casos).  
- Documentar internamente los pasos de la función y proponer bosquejo de API (FastAPI/Flask) para exponerla en producción.

**Estructura del notebook:**  
1. Importar librerías y cargar “columnas_finales.txt” o DataFrame completo  
2. Definir función auxiliar para extraer 14 lags de un `product_id` y `date`  
3. Definir `predict_demand(product_id: str, date: str) -> float`  
4. Probar con ejemplos:  
   - Producto con 14 días de historial completo  
   - Producto con < 14 días de historial (debe devolver `None` o error manejable)  
   - Producto en fecha justo después del último dato de entrenamiento  
5. Documentar en Markdown los pasos internos y proponer idea de API


In [1]:
## 1. Importar librerías y cargar datos necesarios
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import tensorflow as tf
from tensorflow.keras.models import load_model

# 1.1 – Ruta del modelo entrenado de Fase 2
MODEL_PATH = "rnn_demand_model.h5"
DATA_PATH = "DataFinal.csv"

# 1.2 – Cargar DataFrame original de secuencias para tener acceso a 
#       todas las filas con lag features (DataFinal.csv). 
#       Necesitamos este DF para extraer lags históricos hasta 14 días antes de “date”.
DF_SEQ = pd.read_csv(DATA_PATH, parse_dates=["fecha_final_ventana"])



# 1.3 – Cargar lista de columnas finales (en caso de que las tengas en texto)
# Si guardaste columnas_finales.txt, podrías leerlo aquí; 
# de lo contrario, usamos directamente DF_SEQ.columns
lag_cols = [c for c in DF_SEQ.columns if "_t-" in c]  # igual que en Fase 2
lag_cols = sorted(lag_cols)  # opcional: asegúrate de orden por t-1, t-2, …

print("Total de filas en DF_SEQ:", len(DF_SEQ))
print("Número de columnas de lag:", len(lag_cols))


Total de filas en DF_SEQ: 4487025
Número de columnas de lag: 238


In [2]:
def get_14lags_for(product_id: str, date_str: str):
    """
    Dado un product_id y una fecha en 'YYYY-MM-DD', 
    retorna un array de forma (1, 14, n_feats) con dtype=float32, 
    o None si no hay suficiente historial.
    """
    # 2.1 – Convertir date_str a datetime
    fecha_obj = pd.to_datetime(date_str)

    # 2.2 – Filtrar DF_SEQ por este product_id
    df_prod = DF_SEQ[DF_SEQ["product_id"].astype(str) == str(product_id)].copy()
    if df_prod.empty:
        return None  # producto no existe

    # 2.3 – Quedarnos solo con filas cuya fecha_final_ventana < fecha_obj
    df_prod_hist = df_prod[df_prod["fecha_final_ventana"] < fecha_obj]
    if df_prod_hist.empty:
        return None

    # 2.4 – Intentar encontrar la fila justo en fecha_obj - 1 día
    fecha_obj_menos1 = fecha_obj - pd.Timedelta(days=1)
    if fecha_obj_menos1 in set(df_prod_hist["fecha_final_ventana"]):
        fila = df_prod_hist[df_prod_hist["fecha_final_ventana"] == fecha_obj_menos1].iloc[-1]
    else:
        # Si no existe exactamente fecha_obj-1, tomamos la última fila anterior
        fila = df_prod_hist.iloc[-1]

    # 2.5 – Extraer valores de lag_cols como 1D
    arr_1d = fila[lag_cols].values  # tipo original puede ser object

    # 2.6 – Convertir a float32
    arr_1d = arr_1d.astype(np.float32)

    # 2.7 – Reshape a (1, 14, n_feats)
    n_total_feats = len(lag_cols)
    n_timesteps = 14
    n_feats = n_total_feats // n_timesteps
    arr_reshaped = arr_1d.reshape((1, n_timesteps, n_feats))

    return arr_reshaped


In [3]:


# 3.1 – Cargar el modelo una única vez en memoria, sin compilar (compile=False)
_model = load_model(MODEL_PATH, compile=False)

# 3.1.1 – Recompilar el modelo con loss y métricas como strings
_model.compile(
    loss="mse",
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=["mae"]
)

def predict_demand(product_id: str, date: str, verbose: bool = False):
    """
    Parámetros:
      - product_id: ID del producto (cadena o número según cómo esté en DF_SEQ)
      - date: Fecha objetivo en formato 'YYYY-MM-DD'
    Retorno:
      - float: predicción de ventas (sale_amount) para el día “date” 
      - None si no hay suficiente historial o producto no existe
    """
    # 3.2 – Obtener array de lags
    seq_14 = get_14lags_for(product_id, date)
    if seq_14 is None:
        if verbose:
            print(f"No hay suficiente historial para product_id={product_id} antes de {date}.")
        return None

    # 3.3 – Normalización / escalamiento 
    # (Si en Fase 2 aplicaste algún escalador, aquí cargarías y lo aplicarías. 
    #  En este ejemplo asumimos que DF_SEQ ya está normalizado.)

    # 3.4 – Hacer la predicción
    pred = _model.predict(seq_14, verbose=0)

    # 3.5 – Retornar un float redondeado a 2 decimales
    return float(np.round(pred[0, 0], 2))


In [4]:
# 4. Pruebas de predict_demand con ejemplos reales

# 4.1 – Ejemplo 1: producto con historial completo
ej1 = predict_demand(product_id="0", date="2024-06-26", verbose=True)
print("Predicción Ej1 (complete history):", ej1)

# 4.2 – Ejemplo 2: producto con historial corto (<14 días desde su primer registro)
ej2 = predict_demand(product_id="9999", date="2024-06-25", verbose=True)
print("Predicción Ej2 (historial corto):", ej2)

# 4.3 – Ejemplo 3: producto justo después del último dato de entrenamiento
ej3 = predict_demand(product_id="0", date="2024-06-26", verbose=True)
print("Predicción Ej3 (tras última fecha entrenamiento):", ej3)


Predicción Ej1 (complete history): 1.7599999904632568
No hay suficiente historial para product_id=9999 antes de 2024-06-25.
Predicción Ej2 (historial corto): None
Predicción Ej3 (tras última fecha entrenamiento): 1.7599999904632568


In [9]:
ej3 = predict_demand(product_id="8", date="2024-06-29", verbose=True)
print("Predicción Ej3 (tras última fecha entrenamiento):", ej3)

Predicción Ej3 (tras última fecha entrenamiento): 1.6799999475479126


In [12]:
# Cargar un .parquet

df = pd.read_parquet("eval.parquet")
# obtener solo la columna product_id y filtrarlos para que quede unico
unique_product_ids = df['product_id'].unique()
print(f"Número total de product_ids únicos: {len(unique_product_ids)}")
print(f"Primeros 10 product_ids: {unique_product_ids[:10]}")

# Si prefieres como DataFrame
df_unique_products = df[['product_id']].drop_duplicates()
print(f"DataFrame con product_ids únicos tiene {len(df_unique_products)} filas")

Número total de product_ids únicos: 865
Primeros 10 product_ids: [ 38 834 411 686 580 596 740 379   4 600]
DataFrame con product_ids únicos tiene 865 filas


In [13]:
valores = []
for prod in unique_product_ids:
    ej3 = predict_demand(product_id=prod, date="2024-06-29", verbose=True)
    valores.append(ej3)

In [14]:
valores

[1.8799999952316284,
 1.7300000190734863,
 1.8200000524520874,
 1.850000023841858,
 1.559999942779541,
 1.5700000524520874,
 1.3200000524520874,
 0.9800000190734863,
 0.8199999928474426,
 1.7699999809265137,
 1.7100000381469727,
 1.940000057220459,
 1.8899999856948853,
 1.7100000381469727,
 0.8100000023841858,
 0.8500000238418579,
 0.9700000286102295,
 1.0199999809265137,
 1.0299999713897705,
 0.8999999761581421,
 1.1299999952316284,
 1.5,
 1.4800000190734863,
 1.7300000190734863,
 1.9199999570846558,
 1.7899999618530273,
 1.3700000047683716,
 1.059999942779541,
 1.3200000524520874,
 1.1799999475479126,
 1.2300000190734863,
 1.0099999904632568,
 0.9900000095367432,
 1.309999942779541,
 1.059999942779541,
 1.1100000143051147,
 0.9900000095367432,
 0.9900000095367432,
 1.0,
 1.0800000429153442,
 0.9599999785423279,
 0.8600000143051147,
 1.440000057220459,
 1.2799999713897705,
 1.2699999809265137,
 1.159999966621399,
 1.1100000143051147,
 1.7400000095367432,
 1.350000023841858,
 1.3400000

In [15]:
results_df = pd.DataFrame({
    'ProductoID': unique_product_ids,
    'NumSolicitados': valores  
})

# Guardar en CSV
csv_filename = 'predicciones_productos.csv'
results_df.to_csv(csv_filename, index=False)

In [1]:
df = results_df

In [3]:

# 1) Ordenar de mayor a menor por NumSolicitados
df_ordenado = df.sort_values(by="ProductoID", ascending=True)

# 2) (Opcional) Resetear índices tras el ordenamiento
df_ordenado = df_ordenado.reset_index(drop=True)

# 3) Mostrar las primeras filas
print(df_ordenado.head())

   ProductoID  NumSolicitados
0           0            1.76
1           1            2.04
2           2            1.93
3           3            1.11
4           4            0.82


In [4]:
# Suponiendo que df_ordenado es el DataFrame que quieres guardar
ruta_salida = "predicciones.csv"
df_ordenado.to_csv(ruta_salida, index=False, encoding="utf-8")


---

# Aplicacion para API

In [7]:
## 1. Importar librerías y cargar datos necesarios
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import tensorflow as tf
from tensorflow.keras.models import load_model
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

# 1.1 – Ruta del modelo entrenado de Fase 2
MODEL_PATH = "rnn_demand_model.h5"
DATA_PATH = "DataFinal.csv"

app = FastAPI()

# 1.2 – Cargar DataFrame original de secuencias para tener acceso a 
#       todas las filas con lag features (DataFinal.csv). 
#       Necesitamos este DF para extraer lags históricos hasta 14 días antes de “date”.
DF_SEQ = pd.read_csv(DATA_PATH, parse_dates=["fecha_final_ventana"])



# 1.3 – Cargar lista de columnas finales (en caso de que las tengas en texto)
# Si guardaste columnas_finales.txt, podrías leerlo aquí; 
# de lo contrario, usamos directamente DF_SEQ.columns
lag_cols = [c for c in DF_SEQ.columns if "_t-" in c]  # igual que en Fase 2
lag_cols = sorted(lag_cols)  # opcional: asegúrate de orden por t-1, t-2, …

print("Total de filas en DF_SEQ:", len(DF_SEQ))
print("Número de columnas de lag:", len(lag_cols))



def get_14lags_for(product_id: str, date_str: str):
    """
    Dado un product_id y una fecha en 'YYYY-MM-DD', 
    retorna un array de forma (1, 14, n_feats) con dtype=float32, 
    o None si no hay suficiente historial.
    """
    # 2.1 – Convertir date_str a datetime
    fecha_obj = pd.to_datetime(date_str)

    # 2.2 – Filtrar DF_SEQ por este product_id
    df_prod = DF_SEQ[DF_SEQ["product_id"].astype(str) == str(product_id)].copy()
    if df_prod.empty:
        return None  # producto no existe

    # 2.3 – Quedarnos solo con filas cuya fecha_final_ventana < fecha_obj
    df_prod_hist = df_prod[df_prod["fecha_final_ventana"] < fecha_obj]
    if df_prod_hist.empty:
        return None

    # 2.4 – Intentar encontrar la fila justo en fecha_obj - 1 día
    fecha_obj_menos1 = fecha_obj - pd.Timedelta(days=1)
    if fecha_obj_menos1 in set(df_prod_hist["fecha_final_ventana"]):
        fila = df_prod_hist[df_prod_hist["fecha_final_ventana"] == fecha_obj_menos1].iloc[-1]
    else:
        # Si no existe exactamente fecha_obj-1, tomamos la última fila anterior
        fila = df_prod_hist.iloc[-1]

    # 2.5 – Extraer valores de lag_cols como 1D
    arr_1d = fila[lag_cols].values  # tipo original puede ser object

    # 2.6 – Convertir a float32
    arr_1d = arr_1d.astype(np.float32)

    # 2.7 – Reshape a (1, 14, n_feats)
    n_total_feats = len(lag_cols)
    n_timesteps = 14
    n_feats = n_total_feats // n_timesteps
    arr_reshaped = arr_1d.reshape((1, n_timesteps, n_feats))

    return arr_reshaped





# 3.1 – Cargar el modelo una única vez en memoria, sin compilar (compile=False)
_model = load_model(MODEL_PATH, compile=False)

# 3.1.1 – Recompilar el modelo con loss y métricas como strings
_model.compile(
    loss="mse",
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=["mae"]
)

def predict_demand(product_id: str, date: str, verbose: bool = False):
    """
    Parámetros:
      - product_id: ID del producto (cadena o número según cómo esté en DF_SEQ)
      - date: Fecha objetivo en formato 'YYYY-MM-DD'
    Retorno:
      - float: predicción de ventas (sale_amount) para el día “date” 
      - None si no hay suficiente historial o producto no existe
    """
    # 3.2 – Obtener array de lags
    seq_14 = get_14lags_for(product_id, date)
    if seq_14 is None:
        if verbose:
            print(f"No hay suficiente historial para product_id={product_id} antes de {date}.")
        return None

    # 3.3 – Normalización / escalamiento 
    # (Si en Fase 2 aplicaste algún escalador, aquí cargarías y lo aplicarías. 
    #  En este ejemplo asumimos que DF_SEQ ya está normalizado.)

    # 3.4 – Hacer la predicción
    pred = _model.predict(seq_14, verbose=0)

    # 3.5 – Retornar un float redondeado a 2 decimales
    return float(np.round(pred[0, 0], 2))








# Esquema Pydantic para la request
class PredictRequest(BaseModel):
    product_id: str
    date: str  # 'YYYY-MM-DD'

class PredictResponse(BaseModel):
    product_id: str
    date: str
    prediction: float

@app.post("/predict", response_model=PredictResponse)
def api_predict(req: PredictRequest):
    pred = predict_demand(req.product_id, req.date)
    if pred is None:
        raise HTTPException(status_code=404, detail="No hay suficiente historial o producto no existe")
    return PredictResponse(product_id=req.product_id, date=req.date, prediction=pred)

# Para ejecutar: uvicorn Fase3:app --reload --port 8000


ModuleNotFoundError: No module named 'fastapi'